In [2]:
import os
import subprocess

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import *
from sklearn import metrics
from sklearn.metrics import *

In [3]:
#citesc setul de date
datasets = pd.read_csv('ObesityDataSet.csv')

In [4]:
#afisez primele 5 
datasets.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2,3,Sometimes,no,2.0,no,0.0,1,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3,3,Sometimes,yes,3.0,yes,3.0,0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2,3,Sometimes,no,2.0,no,2.0,1,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3,3,Sometimes,no,2.0,no,2.0,0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2,1,Sometimes,no,2.0,no,0.0,0,Sometimes,Public_Transportation,Overweight_Level_I


In [5]:
datasets.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [6]:
le_ObesityLevel = LabelEncoder()
le_ObesityLevel.fit(datasets['NObeyesdad']) 
data_encoded_ObL = le_ObesityLevel.transform(datasets['NObeyesdad'])
data_encoded_ObL

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,

In [7]:
le_ObesityLevel.classes_

array(['Normal_Weight', 'Overweight_Level_I'], dtype=object)

In [8]:
datasets_1 = datasets.apply(le_ObesityLevel.fit_transform) # for all the attributes
datasets_1

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,7,14,29,1,0,1,1,2,0,2,0,0,1,3,3,0
1,0,7,4,19,1,0,2,1,2,1,3,1,4,0,2,3,0
2,1,9,33,46,1,0,1,1,2,0,2,0,3,1,1,3,0
3,1,14,33,57,0,0,2,1,2,0,2,0,3,0,1,4,1
4,1,8,31,60,0,0,1,0,2,0,2,0,0,0,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0,4,19,32,0,1,2,1,2,0,2,0,0,0,2,3,0
495,1,5,33,25,1,1,2,0,0,0,0,1,0,0,3,2,0
496,1,4,25,13,1,1,1,1,2,0,2,0,0,2,2,3,0
497,1,6,8,5,0,0,1,1,2,0,2,0,2,1,2,3,0


In [9]:
X = datasets_1[datasets.columns.drop('NObeyesdad')]
Y = datasets_1['NObeyesdad']

In [10]:
X

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,7,14,29,1,0,1,1,2,0,2,0,0,1,3,3
1,0,7,4,19,1,0,2,1,2,1,3,1,4,0,2,3
2,1,9,33,46,1,0,1,1,2,0,2,0,3,1,1,3
3,1,14,33,57,0,0,2,1,2,0,2,0,3,0,1,4
4,1,8,31,60,0,0,1,0,2,0,2,0,0,0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0,4,19,32,0,1,2,1,2,0,2,0,0,0,2,3
495,1,5,33,25,1,1,2,0,0,0,0,1,0,0,3,2
496,1,4,25,13,1,1,1,1,2,0,2,0,0,2,2,3
497,1,6,8,5,0,0,1,1,2,0,2,0,2,1,2,3


In [11]:
Y

0      0
1      0
2      0
3      1
4      1
      ..
494    0
495    0
496    0
497    0
498    1
Name: NObeyesdad, Length: 499, dtype: int64

In [12]:
#Decision Tree Classification
dt = DecisionTreeClassifier()
scores = cross_val_score(dt, X, Y, cv=5)
print(scores.mean())

0.9578787878787878


In [82]:
#RANDOM FOREST CLASIFICATION

In [83]:
#pentru primul set de parametri
num_trees = 100
max_features = 3
rmodel = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results2 = cross_val_score(rmodel, X, Y, cv=5)
print(results2.mean())

#1. split the dataset: train and test and then fit the classifier using X_train and Y_train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
rmodel.fit(X_train,Y_train)

#2. test the classifier using 3 samples
my_tests = [[0,7,14,29,1,0,1,1,2,0,2,0,0,1,3,3],[0,7,4,19,1,0,2,1,2,1,3,1,4,0,2,3],[1,14,33,57,0,0,2,1,2,0,2,0,3,0,1,4]]
Y_prediction_test= rmodel.predict(my_tests)
print(Y_prediction_test)

#3. print the predictions 
Y_prediction = rmodel.predict(X_test)
print("Y_prediction:",Y_prediction)
print("Y_test:",Y_test)

#4. check/test 5 metrics for classification
#METRICA 1: Matricea de confuzie
#METRICA 2: Accuracy;
#METRICA 3: Precision
#METRICA 4: Recall
#METRICA 5: F1
from sklearn import metrics
matrix = confusion_matrix(Y_test, Y_prediction)
print("Confusion Matrix:")
print(matrix)
print("Accuracy score:",metrics.accuracy_score(Y_test, Y_prediction))
print("Precision score:",metrics.precision_score(Y_test, Y_prediction))
print("Recall score:",metrics.recall_score(Y_test, Y_prediction))
print("F1 score:",f1_score(Y_test, Y_prediction))

0.8877373737373737
[0 0 1]
Y_prediction: [1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0
 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1
 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0]
Y_test: 90     1
254    0
283    0
444    0
474    0
      ..
381    0
56     0
439    0
60     0
208    0
Name: NObeyesdad, Length: 100, dtype: int64
Confusion Matrix:
[[56  4]
 [ 8 32]]
Accuracy score: 0.88
Precision score: 0.8888888888888888
Recall score: 0.8
F1 score: 0.8421052631578948


/home/izabella/anaconda3/envs/my_env/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [13]:
#pentru al doilea set de parametri
num_trees2 = 300
max_features2 = 5
rmodel = RandomForestClassifier(n_estimators=num_trees2, max_features=max_features2)
results2 = cross_val_score(rmodel, X, Y, cv=5)
print(results2.mean())

#1. split the dataset: train and test and then fit the classifier using X_train and Y_train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
rmodel.fit(X_train,Y_train)

#2. test the classifier using 3 samples
my_tests = [[0,7,14,29,1,0,1,1,2,0,2,0,0,1,3,3],[0,7,4,19,1,0,2,1,2,1,3,1,4,0,2,3],[1,14,33,57,0,0,2,1,2,0,2,0,3,0,1,4]]
Y_prediction_test= rmodel.predict(my_tests)
print(Y_prediction_test)

#3. print the predictions
Y_prediction = rmodel.predict(X_test)
print("Y_prediction:",Y_prediction)
print("Y_test:",Y_test)

#4. check/test 5 metrics for classification
#METRICA 1: Matricea de confuzie
#METRICA 2: Accuracy;
#METRICA 3: Precision
#METRICA 4: Recall
#METRICA 5: F1
from sklearn import metrics
matrix = confusion_matrix(Y_test, Y_prediction)
print("Confusion Matrix:")
print(matrix)
print("Accuracy score:",metrics.accuracy_score(Y_test, Y_prediction))
print("Precision score:",metrics.precision_score(Y_test, Y_prediction))
print("Recall score:",metrics.recall_score(Y_test, Y_prediction))
print("F1 score:",f1_score(Y_test, Y_prediction))

0.8937373737373738
[0 0 1]
Y_prediction: [1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0
 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0]
Y_test: 90     1
254    0
283    0
444    0
474    0
      ..
381    0
56     0
439    0
60     0
208    0
Name: NObeyesdad, Length: 100, dtype: int64
Confusion Matrix:
[[57  3]
 [ 6 34]]
Accuracy score: 0.91
Precision score: 0.918918918918919
Recall score: 0.85
F1 score: 0.8831168831168831


/home/izabella/anaconda3/envs/my_env/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [14]:
#EXTRATREES CLASIFICATION

In [15]:
#pentru primul set de parametri
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, Y, cv=5)
print(scores.mean())

#1. split the dataset: train and test and then fit the classifier using X_train and Y_train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
rmodel.fit(X_train,Y_train)

#2. test the classifier using 3 samples
my_tests = [[0,7,14,29,1,0,1,1,2,0,2,0,0,1,3,3],[0,7,4,19,1,0,2,1,2,1,3,1,4,0,2,3],[1,14,33,57,0,0,2,1,2,0,2,0,3,0,1,4]]
Y_prediction_test= rmodel.predict(my_tests)
print(Y_prediction_test)

#3. print the predictions
Y_prediction = rmodel.predict(X_test)
print("Y_prediction:",Y_prediction)
print("Y_test:",Y_test)

#4. check/test 5 metrics for classification
#METRICA 1: Matricea de confuzie
#METRICA 2: Accuracy;
#METRICA 3: Precision
#METRICA 4: Recall
#METRICA 5: F1
from sklearn import metrics
matrix = confusion_matrix(Y_test, Y_prediction)
print("Confusion Matrix:")
print(matrix)
print("Accuracy score:",metrics.accuracy_score(Y_test, Y_prediction))
print("Precision score:",metrics.precision_score(Y_test, Y_prediction))
print("Recall score:",metrics.recall_score(Y_test, Y_prediction))
print("F1 score:",f1_score(Y_test, Y_prediction))

0.8176161616161617
[0 0 1]
Y_prediction: [1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0
 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0]
Y_test: 90     1
254    0
283    0
444    0
474    0
      ..
381    0
56     0
439    0
60     0
208    0
Name: NObeyesdad, Length: 100, dtype: int64
Confusion Matrix:
[[57  3]
 [ 6 34]]
Accuracy score: 0.91
Precision score: 0.918918918918919
Recall score: 0.85
F1 score: 0.8831168831168831


/home/izabella/anaconda3/envs/my_env/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [16]:
#pentru al doilea set de parametri
clf = ExtraTreesClassifier(n_estimators=15, max_depth=None, min_samples_split=5, random_state=0)
scores = cross_val_score(clf, X, Y, cv=5)
print(scores.mean())

#1. split the dataset: train and test and then fit the classifier using X_train and Y_train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
rmodel.fit(X_train,Y_train)

#2. test the classifier using 3 samples
my_tests = [[0,7,14,29,1,0,1,1,2,0,2,0,0,1,3,3],[0,7,4,19,1,0,2,1,2,1,3,1,4,0,2,3],[1,14,33,57,0,0,2,1,2,0,2,0,3,0,1,4]]
Y_prediction_test= rmodel.predict(my_tests)
print(Y_prediction_test)

#3. print the predictions
Y_prediction = rmodel.predict(X_test)
print("Y_prediction:",Y_prediction)
print("Y_test:",Y_test)

#4. check/test 5 metrics for classification
#METRICA 1: Matricea de confuzie
#METRICA 2: Accuracy;
#METRICA 3: Precision
#METRICA 4: Recall
#METRICA 5: F1
from sklearn import metrics
matrix = confusion_matrix(Y_test, Y_prediction)
print("Confusion Matrix:")
print(matrix)
print("Accuracy score:",metrics.accuracy_score(Y_test, Y_prediction))
print("Precision score:",metrics.precision_score(Y_test, Y_prediction))
print("Recall score:",metrics.recall_score(Y_test, Y_prediction))
print("F1 score:",f1_score(Y_test, Y_prediction))

0.8316969696969696
[0 0 1]
Y_prediction: [1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0
 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0]
Y_test: 90     1
254    0
283    0
444    0
474    0
      ..
381    0
56     0
439    0
60     0
208    0
Name: NObeyesdad, Length: 100, dtype: int64
Confusion Matrix:
[[57  3]
 [ 6 34]]
Accuracy score: 0.91
Precision score: 0.918918918918919
Recall score: 0.85
F1 score: 0.8831168831168831


/home/izabella/anaconda3/envs/my_env/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
